In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import torch
from typing import List, Dict
from pprint import pprint

In [2]:
from utils.elasticsearch_utils import get_all_hits

In [3]:
DF_SAVE_PATH = "test.csv"

In [6]:
# Get all data
data: List[Dict] = get_all_hits()

# View all keys
pprint(data[0].keys())

dict_keys(['_index', '_type', '_id', '_score', 'start_time', 'end_time', 'start_timestamp', 'args', 'best_eval', 'last_eval', 'task_info', 'eval_metrics', 'model_summary', 'last_eval_data'])
